In [ ]:
# -*- coding: utf-8 -*-
"""Lista11.ipynb

In [ ]:
Automatically generated by Colaboratory.

In [ ]:
Original file is located at
    https://colab.research.google.com/drive/1L2k4Mqp10rClJXig4cfwda3OVClYoMZh

Aula 6 - Regressão Logística


<br>
# -*- coding: utf8<br>
import numpy as np<br>
import matplotlib.pyplot as plt<br>
from mpl_toolkits.mplot3d import Axes3D<br>
import pandas as pd<br>
from numpy.testing import assert_almost_equal<br>
from numpy.testing import assert_equal<br>
from numpy.testing import assert_array_almost_equal<br>
from numpy.testing import assert_array_equal<br>
# Breast Cancer Dataset

In [ ]:
Esse dataset contém dados de pacientes com tumores de mama, você tentará prever se um tumor é maligno ou benigno de acordo com a espessura do tumor. No gráfico abaixo você pode visualizar os tipos de tumores (1 = maligno, 0 = benigno) por categoria de espessura.

In [ ]:
|Index | Attribute | Domain |
|----|----|----|
0|Sample code number | id number
1|Clump Thickness | 1 - 10
2|Uniformity of Cell Size | 1 - 10
3|Uniformity of Cell Shape | 1 - 10
4|Marginal Adhesion | 1 - 10
5|Single Epithelial Cell Size | 1 - 10
6|Bare Nuclei | 1 - 10
7|Bland Chromatin | 1 - 10
8|Normal Nucleoli | 1 - 10
9|Mitoses | 1 - 10
10|Class | (2 for benign, 4 for malignant)

# Pré-processamento dos dados


<br>
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/'\<br>
                'breast-cancer-wisconsin/breast-cancer-wisconsin.data',<br>
                header=None)<br>
# o nosso dataset contém strings `?` no lugar de dados faltantes<br>
# vamos substituir esses valores para não termos problemas na execução<br>
# das abordagens numéricas<br>
df = df.replace('?', np.NaN)<br>
scaled_df = df.iloc[:, 1:-1].copy()<br>
scaled_df = scaled_df.drop(columns=6)<br>
scaled_df = (scaled_df - scaled_df.mean())/scaled_df.std()<br>
X = scaled_df<br>
X = np.nan_to_num(X)<br>
y = df.iloc[:, -1]<br>
y = y.values<br>
y[y==2] = 0<br>
y[y==4] = 1<br>
nfim, vamos obter os conjuntos de treino e teste para os algoritmos que vamos implementar:


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, \
                                                    random_state=32)


## Regressão Logística<br>
Como discutido na aula, a regressão logística baseia-se na função sigmoide:<br>
$$f(x) = \frac{1}{1+e^{x}}$$<br>
À medida que sua entrada se torna grande e positiva, $f(x)$ se aproxima e se aproxima de 1. À medida que sua entrada se torna grande e negativa, $f(x)$ se aproxima e se aproxima de 0.<br>
### Função Sigmoide<br>
Você consegue implementar a função `sigmoid` abaixo?<br>


In [ ]:
def sigmoid(x):
    # Seu código aqui
    return 1 / (1 + np.exp(-x))


Além disso, $f(x)$ tem a propriedade conveniente que sua derivada é dada por:<br>
$$f'(x) = \frac{e^{-x}}{(1+e^{-x})^2} = f(x)(1-f(x))$$<br>
Vamos testar a função implementada:<br>


In [ ]:
x = np.linspace(-5,5,100)
y = sigmoid(x)
plt.plot(x, y)


### Derivada da Sigmoide<br>
Você consegue implementar a função `sigmoid_prime` abaixo?<br>


In [ ]:
def sigmoid_prime(x):
    # Seu código aqui
    return sigmoid(x) * (1 - sigmoid(x))

In [ ]:
plt.plot(x, sigmoid_prime(x))


Considere que possuímos um conjunto $X_{(n \times k)}$ de $n$ dados - cada dado com $k$ atributos (_features_) - e suas respectivas classes $\mathrm{y}$ - cada classe é zero ou um.<br>
Um par $(\mathbf{x_i}, y_i)$ é um exemplo do nosso conjunto, onde $\mathbf{x_i}$ é a i-ésima linha de $X$ e $y_i$ é sua respectiva classe.<br>
Vamos usar a função sigmoide $f$ para ajustar um modelo de logístico linear:<br>
$$y_i = f(\mathbf{x_i^T}\boldsymbol{\theta}) + \epsilon_i = f_{\boldsymbol{\theta}}(\mathbf{x_i}) + \epsilon_i$$<br>
Note que $\mathbf{x_i^T}\boldsymbol{\theta}$, para $k$ variáveis independentes, nada mais é que o modelo linear visto nas aulas anteriores, que é calculado e dado como entrada para a função logística:<br>
$$\mathbf{x_i^T}\boldsymbol{\theta} = \theta_0 + \theta_1 x_{i1} + \cdots + \theta_k x_{ik}$$<br>
Ao contrário do que acontece no caso da Regressão Linear, precisamos calcular diretamente a função de verossimilhança e seu gradiente. Porém, vimos que é mais simples maximizar o logaritmo da verossimilhança (*log likelihood*):<br>
$$\log ll_{\theta}(y_i~|~\mathbf{x_i}) = y_i \log f_{\boldsymbol{\theta}}(\mathbf{x_i}) + (1-y_i) \log (1-f_{\boldsymbol{\theta}}(\mathbf{x_i}))$$<br>
### Cross Entropy<br>
Ao invés de trabalhar na verossimilhança, vamos invertê-la e utilizar o *cross entropy* para a regressão logística.<br>
$$L(\boldsymbol{\theta}) = -n^{-1}\sum_i \big((1-y_i)\log_2(1-f_{\boldsymbol{\theta}}(\mathbf x_i)) + y_i\log_2(f_{\boldsymbol{\theta}}(\mathbf x_i))\big)$$<br>
A equação acima é a cross-entropy média por observação.<br>
#### Média da Cross Entropy<br>
Você consegue implementar a cross-entropy média na função `cross_entropy_mean` abaixo?<br>
Dica: você pode utilizar a função `np.clip(vetor, limite_inferior, limite_superior)` para evitar imprecisões numéricas quando calcular logarítimos ou realizar divisões.<br>
Nesse caso, você pode usar o `np.clip` para limitar o resultado da função `sigmoid` entre 0.0001 e 0.9999. Assim, se o vetor tiver um valor 1.01 por erro numérico, corrigimos para 0.9999.<br>
```<br>
resultado = sigmoid(vetor)<br>
novo_resultado = np.clip(resultado, 0.00001, 0.99999)<br>
```<br>


In [ ]:
def cross_entropy_mean(X, y, theta):
    z = np.dot(X, theta)
    h = sigmoid(z)
    h = np.clip(h, 0.00001, 0.99999)
    cross_entropy = -(y * np.log(h) + (1 - y) * np.log(1 - h))
    return cross_entropy.mean()


### Derivada da Cross Entropy Média$<br>
A derivada de $L(\theta)$ tem uma forma similar ao da regressão linear. A derivação é similar a que foi apresentada nos [slides](https://docs.google.com/presentation/d/1yGPETPe8o7PPOP6_CF38LHr3vpxgTEnF5LjP-1pkGIc/edit?usp=sharing).<br>
\begin{align*}<br>
  \frac{\partial L(y_i, \mathbf{x_i}, \boldsymbol{\theta})}{\partial \theta_j}<br>
    &=  -\frac{1}{n}\sum_i<br>
      \left(<br>
          (1-y_i)\frac{\partial}{\partial \theta_j}\log_2(1-f_{\boldsymbol{\theta}}(\mathbf x_i)) + y_i\frac{\partial}{\partial \theta_j}\log_2(f_{\boldsymbol{\theta}}(\mathbf x_i))<br>
      \right)\\<br>
    &= -\frac{1}{n}\sum_i<br>
      \left(<br>
          -\frac{(1-y_i)x_{ij}f'(\mathbf{x_i}\boldsymbol{\theta})}{1- f(\mathbf{x_i}\boldsymbol{\theta})} + \frac{y_ix_{ij}f'(\mathbf{x_i}\boldsymbol{\theta}) }{f(\mathbf{x_i}\boldsymbol{\theta})}<br>
      \right)\\<br>
    &= -\frac{1}{n}\sum_i<br>
      \left(<br>
          -\frac{(1-y_i)}{1- f(\mathbf{x_i}\boldsymbol{\theta})} + \frac{y_i}{f(\mathbf{x_i}\boldsymbol{\theta})}<br>
      \right)<br>
        x_{ij}f(\mathbf{x_i}\boldsymbol{\theta})(1 - f(\mathbf{x_i}\boldsymbol{\theta}))\\<br>
    &= -\frac{1}{n}\sum_i<br>
      \left(<br>
          \frac{<br>
            -(1-y_i)f(\mathbf{x_i}\boldsymbol{\theta}) + y_i(1- f(\mathbf{x_i}\boldsymbol{\theta}))<br>
          }{<br>
            f(\mathbf{x_i}\boldsymbol{\theta})(1- f(\mathbf{x_i}\boldsymbol{\theta}))<br>
          }<br>
      \right)<br>
        x_{ij}f(\mathbf{x_i}\boldsymbol{\theta})(1 - f(\mathbf{x_i}\boldsymbol{\theta}))\\<br>
    &= -\frac{1}{n}\sum_i<br>
      \left(<br>
            y_i - f(\mathbf{x_i}\boldsymbol{\theta})<br>
      \right)x_{ij}\\<br>
    &= -\frac{1}{n}\sum_i<br>
      \left(<br>
            y_i - f_\boldsymbol{\theta}(\mathbf{x_i})<br>
      \right)x_{ij}\\<br>
    &= -\frac{1}{n}X^T_j \big(\mathbf{y} - f_\boldsymbol{\theta}(X)\big)<br>
\end{align*}<br>
Logo:<br>
\begin{align*}<br>
  \frac{\partial L(y_i, \mathbf{x_i}, \boldsymbol{\theta})}{\partial \boldsymbol{\theta}}<br>
    &= -\frac{1}{n}X^T \big(\mathbf{y} - f_\boldsymbol{\theta}(X)\big)<br>
\end{align*}<br>
#### Derivada Vetorizada<br>
Você consegue escrever a forma vetorizada da derivada na função `derivadas` abaixo?<br>


In [ ]:
def derivadas(X, y, theta):
    # Seu código aqui
    z = np.dot(X, theta)
    h = sigmoid(z)
    error = h - y
    grad = np.dot(X.T, error) / len(y)
    return grad


## Gradiente Descendente<br>
Na Aula 5, utilizamos o gradiente descendente no contexto da Regressão Linear. Agora, vamos aplicar o gradiente descendente para o caso da Regressão Logística. A ideia aqui é a mesma:<br>
1. Escolha um valor inicial do vetor (e.g., θ = [0.2, 1, -2, 5])<br>
2. Escolha uma taxa de aprendizado (e.g., λ = 0.01)<br>
3. Repita:<br>
  1. Compute a derivada da log-verossimilhança (log likelihood) ll'(θ)<br>
  2. Atualize $\theta$<br>
4. Pare quando convergir<br>
#### Exercício 1<br>
Modifique a implementação de `gd` para o caso da regressão logística.<br>


In [ ]:
def gd(X, y, lambda_=0.01, tol=0.0000001, max_iter=10000):
    theta = np.ones(X.shape[1])
    old_err_sq = np.inf
    i = 0
    while True:
        grad = derivadas(X, y, theta)
        theta_novo = theta - lambda_ * grad
        cross_entropy = cross_entropy_mean(X, y, theta_novo)
        if np.abs(old_err_sq - cross_entropy) <= tol:
            break
        theta = theta_novo
        old_err_sq = cross_entropy
        i += 1
        if i == max_iter:
            break
    return theta


Testando:
<br>
theta = gd(X_train, y_train)<br>
r2_train = cross_entropy_mean(X_train, y_train, theta)<br>
r2_test = cross_entropy_mean(X_test, y_test, theta)<br>
assert_equal(np.round(r2_train,2), np.round(0.08605687702630486,2))<br>
assert_equal(np.round(r2_test,2), np.round(0.18899648326443957,2))<br>
# Gradiente Descendente Estocástico

In [ ]:
Relembrando a Aula 3:
> Muitas vezes usaremos o gradiente descendente para escolher os parâmetros de um modelo de maneira a minimizar alguma noção de erro. Usando a abordagem em lote anterior, cada etapa do método exige que façamos uma previsão e calculemos o gradiente para todo o conjunto de dados, o que faz com que cada etapa demore muito. <br>
> <br> Geralmente, essas funções de erro são *aditivas*, o que significa que o erro preditivo em todo o conjunto de dados é simplesmente a soma dos erros preditivos para cada ponto de dados. <br>
> <br> Quando este é o caso, podemos, em vez disso, aplicar uma técnica chamada gradiente descendente estocástico (ou *stochastic gradient descent*), que calcula o gradiente (e dá um passo) para apenas um ponto por vez. Ele passa sobre os dados repetidamente até atingir um ponto de parada.

# Exercício 2

In [ ]:
Modifique a implementação de `sgd` para o caso da regressão logística.
"""

In [ ]:
def sgd(X, y, lambda_=0.001, tol=0.01, max_iter=10000):
  theta = np.ones(X.shape[1])
  m = len(y)
  for i in range(max_iter):
      r = np.random.randint(m)
      X_r, y_r = np.expand_dims(X[r], axis=0), np.array([y[r]])
      grad = derivadas(X_r, y_r, theta)
      theta = theta - lambda_ * grad
      pred = sigmoid(np.dot(X_r, theta))
      mean_err = np.mean((pred - y_r)**2)
      if mean_err < tol:
          break

In [ ]:
  return theta


Testando:
<br>
theta = sgd(X_train, y_train)<br>
r2_train = cross_entropy_mean(X_train, y_train, theta)<br>
r2_test = cross_entropy_mean(X_test, y_test, theta)<br>
assert_equal(np.round(r2_train,2), np.round(0.09296116832663982,2))<br>
assert_equal(np.round(r2_test,2), np.round(0.19610717861042887,2))<br>
# sklearn: LogisticRegressionCV (somente didatico)

In [ ]:
Por último, temos um exemplo de regressão logística com regularização L2 utilizando a biblioteca sklearn.

In [ ]:
O exemplo abaixo usa a versão com validação cruzada, a função *LogisticRegressionCV*, cuja documentação você encontra [aqui](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html#sklearn.linear_model.LogisticRegressionCV). A maior diferença desta implementação para as abordagens que estudamos na Aula 6 é que a LogisticRegressionCV usa, por padrão, um método da família quasi-Newton invés do gradiente descendente, o [Limited-memory BFGS](https://en.wikipedia.org/wiki/Limited-memory_BFGS).

In [ ]:
Como grande parte das funções implementadas no sklearn, a LogisticRegressionCV possui vários parâmetros configuráveis, cuja melhor configuração vai depender do problema com o qual se está trabalhando. Dê uma olhada na documentação e tente brincar com as várias combinações de parâmetros!
"""

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
clf = LogisticRegressionCV(penalty='l2', cv=3).fit(X_train, y_train)
predictions = clf.predict(X_test)
print(clf.predict_proba(X_test).shape)
# No caso deste modelo, o score padrão computado é a acurácia (0, 1).
# Podemos escolher outras funções já implementadas ou
# passar a nossa própria métrica como parâmetro no momento da construção
# do regressor -> clf = LogisticRegressionCV(..., scoring=metrica_de_interesse)
# métricas implementadas:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
print(clf.score(X_test, y_test))

In [ ]:
clf.coef_

In [ ]:
print("Cross-entropy em teste = ", cross_entropy_mean(X_test, y_test, clf.coef_[0]))